# ⚡ Module 4: Оптимізація структур даних

**Мета:** Зрозуміти, як писати код, що економить гігабайти.

## 1. Interning (Інтернування)

### Small Integer Cache (-5 до 256)
CPython кешує цілі числа від -5 до 256. Це означає, що `a = 100; b = 100` — це **один і той самий** об’єкт.

### String Interning
Python автоматично інтернує рядки, що виглядають як ідентифікатори. Для інших — можна використати `sys.intern()`.

In [1]:
import sys

# === Small Integer Cache ===
a = 256
b = 256
print(f"256 is 256? {a is b}")  # True — cached

a = 257
b = 257
print(f"257 is 257? {a is b}")  # False в REPL (different objects)

a = -5
b = -5
print(f"-5 is -5?  {a is b}")   # True — cached

a = -6
b = -6
print(f"-6 is -6?  {a is b}")   # False — not cached

256 is 256? True
257 is 257? False
-5 is -5?  True
-6 is -6?  False


In [2]:
import sys

# === String Interning ===
# Автоматичне інтернування для рядків-ідентифікаторів
a = "hello"
b = "hello"
print(f"'hello' is 'hello'? {a is b}")  # True — interned

# Рядки з пробілами — не інтернуються
a = "hello world"
b = "hello world"
print(f"'hello world' is 'hello world'? {a is b}")  # Може бути False

# Примусове інтернування
a = sys.intern("hello world")
b = sys.intern("hello world")
print(f"intern('hello world') is intern('hello world')? {a is b}")  # True!

# Практичне застосування: інтернування повторюваних рядків у великих datasets
statuses = [sys.intern("active") for _ in range(100_000)]
print(f"\nВсі 100K 'active' — один об'єкт? {statuses[0] is statuses[-1]}")
print(f"id: {id(statuses[0])} == {id(statuses[-1])}")

'hello' is 'hello'? True
'hello world' is 'hello world'? False
intern('hello world') is intern('hello world')? True

Всі 100K 'active' — один об'єкт? True
id: 2233331365280 == 2233331365280


## 2. List Growth Strategy

Коли ви робите `list.append()`, Python **не** збільшує список на 1 елемент. Він застосовує **овер-алокацію** для amortized O(1) append.

Формула зростання змінювалась між версіями CPython і не є публічним API:
- **Python ≤ 3.12:** `new_size = old_size + (old_size >> 3) + 6` (приблизно)
- **Python 3.13+:** Формулу змінено — конкретні значення відрізняються

Принцип однаковий: для малих списків зростання агресивне (~2x), для великих — більш консервативне (~12.5%). Конкретні resize points залежать від версії — перевірте нижче:

In [3]:
import sys

# Спостерігаємо за зростанням списку
lst = []
prev_size = sys.getsizeof(lst)

print(f"{'Items':>6}  {'Size (bytes)':>12}  {'Allocated slots':>15}  Event")
print("-" * 60)
print(f"{0:>6}  {prev_size:>12}  {'~0':>15}  initial")

resize_points = []
for i in range(30):
    lst.append(i)
    new_size = sys.getsizeof(lst)
    if new_size != prev_size:
        # Приблизна кількість слотів (8 bytes per pointer on 64-bit)
        overhead = 56  # базовий overhead list
        slots = (new_size - overhead) // 8
        resize_points.append(str(slots))
        print(f"{i+1:>6}  {new_size:>12}  {slots:>15}  ↑ RESIZE")
        prev_size = new_size

print(f"\nЗростання слотів: 0 → {' → '.join(resize_points)}")
print("Для малих списків зростання агресивне (2x), для великих — ~12.5% (old + old>>3)")

 Items  Size (bytes)  Allocated slots  Event
------------------------------------------------------------
     0            56               ~0  initial
     1            88                4  ↑ RESIZE
     5           120                8  ↑ RESIZE
     9           184               16  ↑ RESIZE
    17           248               24  ↑ RESIZE
    25           312               32  ↑ RESIZE

Зростання слотів: 0 → 4 → 8 → 16 → 24 → 32
Для малих списків зростання агресивне (2x), для великих — ~12.5% (old + old>>3)


## 3. Dict Internals: Еволюція хеш-таблиць

### До Python 3.6 (Sparse Table)
```
[ключ, значення, None, None, ключ, значення, None, None]  ← 50%+ пустих комірок
```

### Python 3.6+ (Dense Table + Indices)
```
indices: [None, 0, None, 1, None, 2]   ← компактний масив індексів
entries: [(key1, val1), (key2, val2), (key3, val3)]  ← щільний масив
```

Результат: **20–25% економії пам’яті** + збереження порядку вставки.

In [4]:
import sys

# Спостерігаємо за зростанням dict
d = {}
prev_size = sys.getsizeof(d)
print(f"{'Keys':>5}  {'Size (bytes)':>12}  Event")
print("-" * 35)
print(f"{0:>5}  {prev_size:>12}  initial (compact, no hash table)")

for i in range(30):
    d[f"key_{i}"] = i
    new_size = sys.getsizeof(d)
    if new_size != prev_size:
        print(f"{i+1:>5}  {new_size:>12}  ↑ RESIZE")
        prev_size = new_size

print("\nПорожній dict має compact representation (64 bytes).")
print("При першій вставці створюється хеш-таблиця. Далі подвоюється при ~2/3 заповнення.")

 Keys  Size (bytes)  Event
-----------------------------------
    0            64  initial (compact, no hash table)
    1           184  ↑ RESIZE
    6           272  ↑ RESIZE
   11           464  ↑ RESIZE
   22           832  ↑ RESIZE

Порожній dict має compact representation (64 bytes).
При першій вставці створюється хеш-таблиця. Далі подвоюється при ~2/3 заповнення.


## 4. PEP 412: Key-Sharing Dictionaries

Коли кілька екземплярів одного класу мають **однакові ключі** в `__dict__`, Python ділить таблицю ключів між екземплярами (Split Table).

**Нюанс:** `sys.getsizeof()` може показувати більший розмір для instance `__dict__` ніж для звичайного dict, тому що split dict має інший внутрішній layout. Реальна економія видна при великій кількості екземплярів — ключі зберігаються один раз, а не N разів.

In [5]:
import sys
import os
import gc

class SharedKeys:
    def __init__(self):
        self.a = 1
        self.b = 2
        self.c = 3

# Щоб побачити реальну економію PEP 412 — порівнюємо RSS для великої кількості об'єктів
try:
    import psutil
    def get_rss():
        return psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024
except ImportError:
    get_rss = None

# Порівняємо: N об'єктів з class __dict__ vs N звичайних dictionaries
N = 200_000

if get_rss:
    gc.collect()
    baseline = get_rss()

    # Тест 1: Key-Sharing dicts (через class instances)
    instances = [SharedKeys() for _ in range(N)]
    mem_shared = get_rss()

    del instances
    gc.collect()
    mid = get_rss()

    # Тест 2: Regular dicts (ключі дублюються в кожному dict)
    regular_dicts = [{"a": 1, "b": 2, "c": 3} for _ in range(N)]
    mem_regular = get_rss()

    del regular_dicts
    gc.collect()

    shared_mb = mem_shared - baseline
    regular_mb = mem_regular - mid
    print(f"200K class instances (Key-Sharing):  +{shared_mb:.1f} MB")
    print(f"200K regular dicts (no sharing):     +{regular_mb:.1f} MB")
    if regular_mb > 0:
        print(f"Економія PEP 412: {regular_mb / shared_mb:.1f}x менше пам'яті")
else:
    print("psutil потрібен для цієї демонстрації")

print("\nPEP 412: екземпляри одного класу ділять таблицю ключів.")
print("sys.getsizeof() не показує цю економію — вона видна лише на рівні RSS.")

200K class instances (Key-Sharing):  +20.5 MB
200K regular dicts (no sharing):     +35.9 MB
Економія PEP 412: 1.8x менше пам'яті

PEP 412: екземпляри одного класу ділять таблицю ключів.
sys.getsizeof() не показує цю економію — вона видна лише на рівні RSS.


## 5. `__slots__` vs `__dict__`: Бенчмарк на 500K об’єктах

`__slots__` змушує Python зберігати атрибути в фіксованому C-масиві замість словника.

In [6]:
import os
import gc
import sys

try:
    import psutil
    def get_memory_usage():
        return psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024
except ImportError:
    print("psutil not found, using approximate measurement")
    import tracemalloc
    tracemalloc.start()
    def get_memory_usage():
        return tracemalloc.get_traced_memory()[0] / 1024 / 1024

class DictBased:
    def __init__(self, id, status, payload):
        self.id = id
        self.status = status
        self.payload = payload

class SlotsBased:
    __slots__ = ('id', 'status', 'payload')
    def __init__(self, id, status, payload):
        self.id = id
        self.status = status
        self.payload = payload

N = 500_000
static_status = sys.intern("active")
static_payload = sys.intern("payload_data")

print(f"Base Memory: {get_memory_usage():.2f} MB")

# --- ТЕСТ 1: __dict__ ---
print("\n--- Phase 1: Dict-based objects ---")
data_dicts = [DictBased(i, static_status, static_payload) for i in range(N)]
mem_after_dicts = get_memory_usage()
print(f"Memory with Dicts: {mem_after_dicts:.2f} MB")

del data_dicts
_ = gc.collect()  # suppress output
base_mem = get_memory_usage()
print(f"Memory cleared: {base_mem:.2f} MB")

# --- ТЕСТ 2: __slots__ ---
print("\n--- Phase 2: Slots-based objects ---")
data_slots = [SlotsBased(i, static_status, static_payload) for i in range(N)]
mem_after_slots = get_memory_usage()
print(f"Memory with Slots: {mem_after_slots:.2f} MB")

# --- РЕЗУЛЬТАТИ ---
diff_dict = mem_after_dicts - base_mem
diff_slots = mem_after_slots - base_mem

print("\n" + "=" * 30)
print(f"Dicts took: ~{diff_dict:.2f} MB")
print(f"Slots took: ~{diff_slots:.2f} MB")
if diff_slots > 0:
    print(f"Savings: {diff_dict / diff_slots:.1f}x")

del data_slots
_ = gc.collect()

Base Memory: 82.64 MB

--- Phase 1: Dict-based objects ---
Memory with Dicts: 147.38 MB
Memory cleared: 84.55 MB

--- Phase 2: Slots-based objects ---
Memory with Slots: 132.04 MB

Dicts took: ~62.83 MB
Slots took: ~47.49 MB
Savings: 1.3x


## 6. Mutable Default Arguments: Анти-патерн

Класична пастка Python: мутабельний аргумент за замовчуванням створюється **один раз** і перевикористовується.

In [7]:
# НЕБЕЗПЕЧНО!
def add_log(msg, logs=[]):
    print(f"ID of list: {id(logs)}")
    logs.append(msg)
    return logs

print(add_log("Error 1"))
print(add_log("Error 2"))
print(add_log("Error 3"))
print("\n❗ Один і той самий список! Дані накопичуються.")

ID of list: 2233375548672
['Error 1']
ID of list: 2233375548672
['Error 1', 'Error 2']
ID of list: 2233375548672
['Error 1', 'Error 2', 'Error 3']

❗ Один і той самий список! Дані накопичуються.


In [8]:
# ПРАВИЛЬНО:
def add_log_safe(msg, logs=None):
    if logs is None:
        logs = []
    logs.append(msg)
    return logs

print(add_log_safe("Error 1"))
print(add_log_safe("Error 2"))
print("Тепер кожний виклик отримує новий список!")

['Error 1']
['Error 2']
Тепер кожний виклик отримує новий список!


## 7. gc.freeze(): Оптимізація для Long-Lived об’єктів

In [9]:
import gc

def print_stats(step):
    counts = gc.get_count()
    # gc.get_count() повертає (count0, count1, count2):
    # count0 = алокації мінус деалокації з останнього збору Gen 0
    # count1 = кількість зборів Gen 0 з останнього збору Gen 1
    # count2 = кількість зборів Gen 1 з останнього збору Gen 2
    print(f"[{step}] gc.get_count(): {counts}")

print("--- START ---")
data = [dict(id=i, val="x"*100) for i in range(50000)]
print_stats("After Creation")

_ = gc.collect()
_ = gc.collect()
print_stats("After pushing to Gen 2")

gc.freeze()
print_stats("After gc.freeze()")
print(f"Data still accessible? {len(data)} items.")

new_trash = [i for i in range(100)]
print_stats("After new trash")
print("\nFrozen об'єкти не скануються GC → менше оверхеду.")

--- START ---
[After Creation] gc.get_count(): (376, 0, 0)
[After pushing to Gen 2] gc.get_count(): (4, 0, 0)
[After gc.freeze()] gc.get_count(): (3, 0, 0)
Data still accessible? 50000 items.
[After new trash] gc.get_count(): (4, 0, 0)

Frozen об'єкти не скануються GC → менше оверхеду.
